In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict
from IPython.display import display, Markdown

In [48]:
import json
import os
import requests

In [49]:
load_dotenv(override=True)

True

In [50]:
@function_tool
def get_current_date() -> str:
    """
    Get the current date in ISO format (YYYY-MM-DD) to calculate video recency.
    """
    from datetime import datetime
    current_date = datetime.now().strftime("%Y-%m-%d")
    return f"Current date: {current_date}"


@function_tool
def search_tool(query: str) -> str:
    """
    Useful to search the internet about a given topic and return the results
    """
    top_result_to_return = 10
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    
    try:
        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        if 'organic' not in data or not data['organic']:
            return "No organic search results found. Try refining your query."
            
        results = data['organic']
        string = []
        for result in results[:top_result_to_return]:
            try:
                snippet = result.get('snippet', 'No snippet available')
                title = result.get('title', 'No title available')
                link = result.get('link', 'No link available')
                
                string.append('\n'.join([
                    f"Title: {title}", 
                    f"Link: {link}",
                    f"Snippet: {snippet}", 
                    "\n--------------"
                ]))
            except KeyError:
                continue
        
        return '\n'.join(string)
        
    except requests.exceptions.RequestException as e:
        return f"Search error: {str(e)}"

@function_tool
def video_search_tool(query: str) -> str:
    """
    Search for video content related to a given topic and return the results.
    """
    top_result_to_return = 8  # Increased to get more options for filtering
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query, "type": "videos"})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    
    try:
        response = requests.request("POST", url, headers=headers, data=payload, timeout=10)
        response.raise_for_status()
        
        data = response.json()
        if 'videos' not in data or not data['videos']:
            return "No video results found. Try refining your query."
        
        results = data['videos']
        string = []
        for result in results[:top_result_to_return]:
            try:
                # Extract duration and convert to seconds for filtering
                duration_str = result.get('duration', '0:00')
                
                # Skip videos less than 3 minutes (180 seconds)
                minutes, seconds = 0, 0
                if ':' in duration_str:
                    parts = duration_str.split(':')
                    if len(parts) == 2:
                        minutes, seconds = int(parts[0]), int(parts[1])
                    elif len(parts) == 3:
                        hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
                        minutes += hours * 60
                
                total_seconds = minutes * 60 + seconds
                if total_seconds < 180:  # Skip videos shorter than 3 minutes
                    continue
                
                # Include all available fields in the output
                output_elements = [
                    f"Title: {result.get('title', 'No title available')}",
                    f"Link: {result.get('link', 'No link available')}",
                    f"Channel: {result.get('channel', 'Unknown channel')}",
                    f"Duration: {duration_str}",
                    f"Published: {result.get('date', 'Unknown date')}",
                    f"Snippet: {result.get('snippet', 'No snippet available')}",
                    "\n--------------"
                ]
                
                string.append('\n'.join(output_elements))
            except KeyError:
                continue
        
        return '\n'.join(string)
    
    except requests.exceptions.RequestException as e:
        return f"Video search error: {str(e)}"

In [51]:
# Web Search Agent
web_search_agent = Agent(
    name="web_search_agent",
    instructions="""You are a sophisticated research assistant specializing in comprehensive web content analysis. 
    
    Your task:
    1. Search the web for the given query term
    2. Analyze multiple sources to identify factual, high-quality information
    3. Synthesize findings into a concise, information-dense summary
    4. Include relevant statistics, trends, key players, and recent developments when available
    5. Prioritize authoritative sources (academic, industry leaders, reputable publications)
    6. Note conflicting viewpoints or controversies if they exist
    
    Format requirements:
    - Write 4-5 detailed paragraphs 
    - Use neutral, factual language
    - Include proper citations for key facts
    - Structure information from most important to least important
    - Focus exclusively on information directly relevant to the query
    
    This will be integrated into a comprehensive research report, so accuracy, conciseness and depth are critical. Avoid promotional content, subjective claims, and fluff. Do not include personal commentary.""",
    model="gpt-4o-mini",
    tools=[search_tool],
    model_settings=ModelSettings(tool_choice="required")
)

# Video Search Agent
video_search_agent = Agent(
    name="video_search_agent",
    instructions="""You are an elite video content curator specializing in identifying the highest-quality educational and informational videos.
    IMPORTANT: Always start by checking the current date using the get_current_date tool before evaluating videos.

    Your primary responsibilities:
    First, call the get_current_date tool to determine today's date
    1. Analyze each video search result thoroughly, focusing on:
       - Duration (prioritize comprehensive content 10+ minutes, ignore videos under 3 minutes)
       - Channel reputation and expertise (academic institutions, recognized industry experts)
       - Content depth based on title and snippet
       - Recency and relevance (prioritize newer content unless historical context is valuable)
       - Educational value over entertainment
    
    2. For each high-quality video you recommend, provide:
       - Complete title with direct, clickable link
       - Channel name with assessment of their expertise
       - Duration (longer form content, ideally 10+ minutes, is preferred)
       - Publication date (prioritize recent content)
       - A 1-2 sentence summary extracted from the snippet
       - Why this specific video is valuable (unique insights, comprehensive coverage, expert analysis)
    
    3. Selection criteria:
       - Select a diverse range of high-quality videos (4-6 videos)
       - Prioritize content from reputable educational channels, industry experts, and thought leaders
       - Include a mix of formats (lectures, tutorials, interviews, case studies)
       - Focus on videos that provide substantial, in-depth knowledge
       - Avoid promotional content, clickbait titles, or content lacking substantial information
       - Recency is a primary factor 
       - Content from the past 3 months should be given highest priority when available and relevant
       - Older content (6+ months) should only be included if it offers exceptional value not found in recent videos
       - For rapidly evolving topics, limit recommendations to content less than 90 days old
    
    4. Structure your response as a curated list with clear formatting:
       - List videos in order of quality/relevance
       - Group by subtopics if appropriate
       - Highlight what makes each video especially valuable
    
    Remember: Your goal is to save researchers time by identifying only the most substantive, informative videos from recognized experts. Quality over quantity.""",
    model="gpt-4o-mini",
    tools=[video_search_tool,get_current_date],
    model_settings=ModelSettings(tool_choice="required")
)

In [106]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(video_search_agent, message)

display(Markdown(result.final_output))

Here is a curated list of high-quality videos on the latest AI agent frameworks in 2025. Each of these selections is chosen based on expert insights, depth of content, and recency:

### 1. [The Definitive Guide to Building AI Agents in 2025](https://www.youtube.com/watch?v=77nHShlpCfQ)
   - **Channel:** AI Software Developers
   - **Duration:** 32:45
   - **Published:** January 6, 2025
   - **Summary:** This video provides an overview of what AI agents are, how they differ from traditional workflows, and lays out composable patterns. 
   - **Value:** It's a comprehensive guide emphasizing foundational knowledge for anyone looking to develop AI agents effectively.

### 2. [2025 AI AGENT Masterclass - Learn How To Build ANYTHING](https://www.youtube.com/watch?v=HkFDWwmtZ-M&pp=0gcJCfcAhR29_xXO)
   - **Channel:** All About AI
   - **Duration:** 44:50
   - **Published:** January 5, 2025
   - **Summary:** This masterclass focuses on using large language models (LLMs) to build versatile AI agents.
   - **Value:** The detailed structure covers everything from the basics to advanced applications, making it ideal for both beginners and seasoned developers.

### 3. [How to Build Effective AI Agents In 2025 (without the hype)](https://www.youtube.com/watch?v=I7synA1L4aw)
   - **Channel:** The Codebender
   - **Duration:** 9:52
   - **Published:** 1 month ago
   - **Summary:** This tutorial explains how to create AI agents using the DeepSeek framework, focusing on practical applications rather than hype.
   - **Value:** A concise, focused approach makes it valuable for developers looking to quickly grasp practical AI agent creation without marketing fluff.

### 4. [Agent Development Kit (ADK) Masterclass: Build AI Agents & Automate Workflows](https://www.youtube.com/watch?v=P4VFL9nIaIA)
   - **Channel:** aiwithbrandon
   - **Duration:** 3:12:48
   - **Published:** 24 hours ago
   - **Summary:** This masterclass provides a thorough examination of the Agent Development Kit, guiding viewers from beginner to pro in building AI agents.
   - **Value:** The extensive duration and depth of content make it an exceptional resource for in-depth learning about AI agents.

### 5. [How I'd Learn To Build AI Agents in 2025 (if I could start over)](https://www.youtube.com/watch?v=lgMkZxkOgbI)
   - **Channel:** Ritesh Verma
   - **Duration:** 17:17
   - **Published:** February 16, 2025
   - **Summary:** While offering personal insights, the video outlines the most effective paths for learning AI agent development.
   - **Value:** This video provides a mentor-like perspective for newcomers to understand the landscape of AI agent frameworks.

### 6. [AI Agents Will Dominate 2025! Here’s Why & How!](https://www.youtube.com/watch?v=OHwaC1sSBTM&pp=0gcJCfcAhR29_xXO)
   - **Channel:** Bankless
   - **Duration:** 1:24:36
   - **Published:** January 2, 2025
   - **Summary:** This video discusses various frameworks, including Ai16z's Eliza, focusing on how they could transform multi-chain interactions in 2025.
   - **Value:** Offers a forward-looking perspective on the potential of AI agent frameworks in the blockchain domain, making it highly relevant for tech enthusiasts.

These videos provide a rich and diverse understanding of the AI agent landscape in 2025, catering to both newcomers and experienced developers looking to enhance their knowledge.

In [52]:
class WebSearchItem(BaseModel):
    reason: str 
    """Your reasoning for why this search is important to the query."""

    query: str 
    """The search term to use for the web search."""
    
    search_type: str
    """The type of search to perform: 'web' or 'video'."""
    
    priority: int 
    """Priority of this search on a scale of 1-10, with 10 being highest priority."""

    

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] 
    """A list of web and video searches to perform to best answer the query."""
    
    strategy: str 
    """The overall search strategy explaining the approach to answering the query."""
    
    


# Balanced Research Planner
# planner_agent = Agent(
#     name="BalancedResearchPlanner",
#     instructions="""You are a research strategist who creates BALANCED, COMPREHENSIVE search plans.

# CORE PRINCIPLES:
# 1. Consider ALL valuable information types without bias toward any particular category
# 2. Include a BROAD SPECTRUM of search approaches based on what would genuinely illuminate the topic
# 3. Balance depth and breadth across different knowledge dimensions

# For EVERY research query, thoughtfully consider these DIVERSE INFORMATION DIMENSIONS:

# - CONCEPTUAL: Theoretical foundations, principles, frameworks, models
# - PRACTICAL: Applications, implementations, use cases, real-world examples
# - INSTRUCTIONAL: Tutorials, guides, how-tos, documentation, code examples
# - TECHNICAL: Specifications, architectures, infrastructures, methodologies
# - COMPARATIVE: Benchmarks, alternatives, pros/cons, competitive landscape
# - CONTEXTUAL: Historical evolution, current state, future directions
# - CRITICAL: Limitations, challenges, debates, controversies, ethical concerns
# - SOCIAL: Community perspectives, expert opinions, user experiences

# PLANNING APPROACH:
# 1. Analyze which of these dimensions would provide the most valuable insights for THIS SPECIFIC QUERY
# 2. Include searches across MULTIPLE dimensions that complement each other
# 3. Ensure appropriate balance between different information types
# 4. Create search terms that target high-value information within each selected dimension

# SEARCH CONSTRUCTION PRINCIPLES:
# - Create genuinely distinct searches that explore different facets
# - Use precise, targeted language for each search
# - Consider both mainstream and specialized sources
# - Include both established knowledge and emerging developments
# - Balance between conceptual understanding and practical application

# FOR EACH SEARCH:
# 1. Create an effective, focused search term
# 2. Choose web or video based on which format would best deliver this specific information
# 3. Explain the dimension this search explores and its unique contribution
# 4. Assign appropriate priority level

# Your goal is to create a thoughtfully balanced set of searches that explores all valuable dimensions of the topic without overemphasizing or neglecting any important aspect.""",
#     model="gpt-4o-mini",
#     output_type=WebSearchPlan
# )

# Improved Planner Agent
planner_agent = Agent(
    name="ResearchPlannerAgent",
    instructions="""You are an expert research strategist creating highly effective search queries for comprehensive research.

Analyze the query and develop 4-7 DIVERSE and COMPLEMENTARY search queries that cover DIFFERENT DIMENSIONS of the topic:

REQUIRED QUERY CATEGORIES (include at least one from each category):
1. TUTORIAL/HOW-TO: Include at least one search for tutorials, guides, or implementation examples
   - Example: "building AI agent tutorial" or "how to implement reinforcement learning"

2. TECHNICAL/ACADEMIC: Include at least one search for technical specifications or academic papers
   - Example: "quantum computing decoherence technical details" or "transformer architecture explained"

3. COMPARISON/EVALUATION: Include at least one search comparing different approaches or frameworks
   - Example: "comparing JavaScript frameworks 2025" or "LLM benchmarks comparison"

4. PRACTICAL/APPLICATION: Include at least one search focused on real-world implementations
   - Example: "AI agent practical applications" or "implementing federated learning examples"

ADDITIONAL CATEGORIES (include as appropriate):
- CODE EXAMPLES: For programming/technical topics, include a search for code samples
- CHALLENGES/LIMITATIONS: Search for problems and constraints in the field
- CASE STUDIES: Search for real-world examples and outcomes
- EXPERT OPINIONS: Search for thought leaders' perspectives and predictions
- HISTORICAL CONTEXT: Search for evolution and development of the topic

VIDEO SEARCH REQUIREMENTS:
- Always include at least 3 video search for topics that benefit from visual explanation
- Target specific formats (tutorials, demonstrations, talks) in video searches
- Use specific video query syntax (e.g., "step-by-step tutorial video")

CRITICAL QUERY GUIDELINES:
- AVOID adding years to every query (don't use specific dates in all searches)
- Make each search term TRULY DISTINCT from others (no near-duplicates)
- Use 3-6 words per search; be precise and focused
- For recency, use "latest", "recent", "new" rather than specific years
- Vary search syntax across queries (questions, commands, phrases)
- Include some technical/specific terms in appropriate queries

FOR EACH SEARCH:
1. Create a truly unique search term following the category guidelines above
2. Assign appropriate type (web or video)
3. Provide a clear reason why this SPECIFIC search adds unique value
4. Assign priority (1-10) based on information importance

Let the complexity of the topic determine the exact number of searches needed. and also balance the web and video searches but more focus on web searches""",
    model="gpt-4o-mini",
    output_type=WebSearchPlan
)

In [23]:
message = "Latest AI Agent framework in 2025"
with trace("search"):
    result = await Runner.run(planner_agent,message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find the most current tutorials on developing AI agents, which can help beginners understand the implementation process.', query='AI agent development tutorial', search_type='web', priority=8), WebSearchItem(reason='This search will provide access to the latest academic papers and technical specifications about AI agent frameworks that can inform best practices and methodologies.', query='AI agent framework technical papers', search_type='web', priority=9), WebSearchItem(reason='Comparing different AI agent frameworks will allow for a nuanced understanding of their strengths and weaknesses, helping developers make informed choices.', query='comparison of AI agent frameworks', search_type='web', priority=7), WebSearchItem(reason='Searching for practical applications of AI agents will showcase real-world cases, helping to understand how they are utilized across various industries.', query='real-world applications of AI agents', search_type='web', priori

In [53]:
class RecommendedVideo(BaseModel):
    title: str
    """Title of the recommended video."""
    
    link: str
    """Direct clickable link to the video."""
    
    description: str
    """Brief description of what the video covers and why it's valuable."""
    
    creator: str
    """Name of the video creator or channel."""
    
    

class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report in markdown format."""
    
    recommended_videos: list[RecommendedVideo]
    """List of recommended videos with complete details."""

    follow_up_questions: list[str]
    """Suggested topics to research further."""
    
    key_insights: list[str]
    """List of the most important takeaways from the research."""
    
    
writer_agent = Agent(
    name="DeepResearchWriter",
    instructions="""
"You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 3000 words."
""",
model="gpt-4o",
output_type=ReportData
)

In [54]:
async def plan_searches(query: str):
    """Use the planner_agent to plan which searches to run for the query"""
    print("Planning Searches...")
    result = await Runner.run(planner_agent, f"query: {query}")
    
    # Sort searches by priority (highest first)
    sorted_searches = sorted(
        result.final_output.searches, 
        key=lambda s: s.priority if hasattr(s, 'priority') else 5,
        reverse=True
    )
    
    web_searches = [s for s in sorted_searches if s.search_type == 'web']
    video_searches = [s for s in sorted_searches if s.search_type == 'video']
    
    print(f"Will perform {len(web_searches)} web searches and {len(video_searches)} video searches")
    
    # Print the planned searches with priorities
    for i, search in enumerate(sorted_searches):
        priority = getattr(search, 'priority', 5)
        print(f"  {i+1}. [{search.search_type.upper()}] (Priority: {priority}) {search.query}")
        print(f"     Reason: {search.reason}")
    
    return sorted_searches

async def perform_web_search(item: WebSearchItem):
    """Use the web search agent to run a web search"""
    priority = getattr(item, 'priority', 5)
    print(f"Searching web for: {item.query} (Priority: {priority})...")
    
    input_text = f"search query: {item.query}\nreason for searching: {item.reason}"
    result = await Runner.run(web_search_agent, input_text)
    
    return {
        "type": "web",
        "query": item.query,
        "reason": item.reason,
        "priority": priority,
        "result": result.final_output
    }

async def perform_video_search(item: WebSearchItem):
    """Use the video search agent to run a video search"""
    priority = getattr(item, 'priority', 6)
    print(f"Searching videos for: {item.query} (Priority: {priority})...")
    
    input_text = f"search query: {item.query}\nreason for searching: {item.reason}"
    result = await Runner.run(video_search_agent, input_text)
    
    return {
        "type": "video",
        "query": item.query,
        "reason": item.reason,
        "priority": priority,
        "result": result.final_output
    }

async def perform_searches(searches: list[WebSearchItem]):
    """Execute all planned searches based on their type"""
    print("Performing Searches...")
    
    tasks = []
    for item in searches:
        if item.search_type == 'web':
            tasks.append(asyncio.create_task(perform_web_search(item)))
        elif item.search_type == 'video':
            tasks.append(asyncio.create_task(perform_video_search(item)))
    
    results = await asyncio.gather(*tasks)
    print("All searches completed")
    return results



async def write_report(query: str, search_results: list):
    """Use the writer agent to write a comprehensive deep research report using raw search results"""
    print("Creating comprehensive research report...")
    
    # Separate web and video results
    web_results = [r for r in search_results if r["type"] == "web"]
    video_results = [r for r in search_results if r["type"] == "video"]
    
    # Format the raw results from web searches
    web_text = "\n\n## WEB SEARCH RESULTS:\n\n"
    for i, res in enumerate(web_results, 1):
        web_text += f"### Web Search {i}: '{res['query']}'\n"
        web_text += f"**Priority**: {res.get('priority', 'N/A')}\n"
        web_text += f"**Research Purpose**: {res['reason']}\n\n"
        web_text += f"**Raw Results**:\n```\n{res['result']}\n```\n\n"
    
    # Format the raw results from video searches
    video_text = "\n\n## VIDEO SEARCH RESULTS:\n\n"
    for i, res in enumerate(video_results, 1):
        video_text += f"### Video Search {i}: '{res['query']}'\n"
        video_text += f"**Priority**: {res.get('priority', 'N/A')}\n"
        video_text += f"**Research Purpose**: {res['reason']}\n\n"
        video_text += f"**Raw Results**:\n```\n{res['result']}\n```\n\n"
    
    # Create research instructions with complete context
    input_text = f"""# COMPREHENSIVE RESEARCH ASSIGNMENT

## Research Query:
{query}

## Your Task:
Create a comprehensive, authoritative report on this topic synthesizing all the provided search results.

{web_text}
{video_text}

## Report Guidelines:
1. Structure your report with clear sections and subsections
2. Start with an executive summary of key findings
3. Create a proper table of contents
4. Analyze and synthesize all provided information
5. Create a visually organized document with proper markdown formatting
6. Write a substantial report (2500-3000+ words) with appropriate depth for the topic
7. Recommend high-quality video resources where relevant

## FORMAT:
- Use markdown headers, lists, and emphasis appropriately
- Include a table of contents with links to sections
- Create comparative tables where useful
- Format key technical details, statistics, and metrics for clarity
- Use sub-sections to organize complex topics

This is the definitive report on this topic - make it comprehensive, accurate, and valuable.
"""
    
    print("Analyzing all raw search data...")
    result = await Runner.run(writer_agent, input_text)
    print("Completed comprehensive research report")
    return result.final_output

In [55]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print(f"Starting research on: {query}")
    
    # Step 1: Plan searches
    search_plan = await plan_searches(query)
    
    # Step 2: Execute searches
    search_results = await perform_searches(search_plan)
    
    # Step 3: Generate report
    report = await write_report(query, search_results)
    
    print("Research complete!")
    print(report.short_summary)
    # Save report to markdown file
    with open("research_report.md", "w") as f:
        f.write(report.markdown_report)

Starting research on: Latest AI Agent frameworks in 2025
Planning Searches...
Will perform 5 web searches and 2 video searches
  1. [WEB] (Priority: 8) latest AI agent framework specifications
     Reason: This technical search will offer in-depth insights into the latest specifications and developments in AI agent frameworks, beneficial for understanding their architecture.
  2. [WEB] (Priority: 7) how to create AI agents using modern frameworks
     Reason: This tutorial search will provide practical guidance on how to build AI agents using the latest frameworks.
  3. [WEB] (Priority: 7) real-world applications of AI agent frameworks
     Reason: This practical application search will uncover real-world use cases of AI agent frameworks, highlighting their implementation in various industries.
  4. [WEB] (Priority: 6) comparison of AI agent frameworks 2025
     Reason: This comparison will help identify the strengths and weaknesses of different AI agent frameworks, which is crucial fo

In [11]:
query = "What are the latest advancements in quantum computing?"
print(f"Starting research on: {query}")
search_plan = await plan_searches(query)

search_results = await perform_searches(search_plan)

report = await write_report(query, search_results)

    # Print report summary
print("\n--- REPORT SUMMARY ---")
print(report.short_summary)
    
    # Save report to markdown file
with open("research_report.md", "w") as f:
    f.write(report.markdown_report)
    
print(f"\nFull report saved to research_report.md")

Starting research on: What are the latest advancements in quantum computing?
Planning Searches...
Will perform 5 web searches and 2 video searches
  1. [WEB] (Priority: 10) latest advancements in quantum computing 2023
     Reason: To understand the overall landscape and recent developments in quantum computing.
  2. [WEB] (Priority: 9) quantum computing breakthroughs technologies 2023
     Reason: To get insights into the specific breakthroughs and technologies being developed in quantum computing.
  3. [WEB] (Priority: 8) applications of quantum computing advancements 2023
     Reason: To explore the implications and potential applications of recent quantum computing advancements.
  4. [WEB] (Priority: 7) top quantum computing researchers companies 2023
     Reason: To identify leading researchers and organizations driving advancements in quantum computing.
  5. [VIDEO] (Priority: 6) quantum computing advancements video 2023
     Reason: To find visual explanations and discussions re